In [ ]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('HF_API_KEY'))

import os
import requests
import re
!pip install wandb==0.19.10 -q

!pip install flash-attn
!pip install -U flash-attn bitsandbytes transformers accelerate
# Install unsloth and other dependencies
!pip install "unsloth[colab-newest-deps] @ git+https://github.com/unslothai/unsloth.git"
!pip install -U vllm --pre --extra-index-url https://wheels.vllm.ai/7489ec0bab2904dcc4001af59a942a16756fdbbc
f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
with open("vllm_requirements.txt", "wb") as file:
    file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
!pip install -r vllm_requirements.txt

# Skip restarting message in Colab
!pip install accelerate xformers==0.0.29.post3 peft==0.15.2 trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.6 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-

Ignoring importlib_metadata: markers 'python_version < "3.10"' don't match your environment
Ignoring six: markers 'python_version > "3.11"' don't match your environment
Ignoring setuptools: markers 'python_version > "3.11"' don't match your environment
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: compressed-tensors
    Found existing installation: compressed-tensors 0.9.3
    Uninstalling compressed-tensors-0.9.3:
      Successfully uninstalled compressed-tensors-0.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.8.5.post1 requires compressed-tensors==0.9.3, but you have compressed-tensors 0.9.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
import os
import re
import time
import requests
import concurrent.futures # Crucial for concurrency
from typing import List, Dict, Any, Optional
import torch

# --- Environment Variables & Constants (Keep these as before) ---
GEMINI_EVAL_MODEL_NAME = "gemini-2.5-pro-preview-05-06"
GEMINI_API_URL_TEMPLATE = "https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent"
DEEPSEEK_EVAL_MODEL_NAME = "deepseek-chat"
DEEPSEEK_API_URL = "https://api.deepseek.com/chat/completions"

# --- Evaluator Classes (BaseEvaluator, GeminiConcurrentUserEvaluator, DeepseekConcurrentUserEvaluator) ---
class BaseEvaluator:
    def __init__(self, api_key_env_var: str, model_name: str):
        self.model_name = model_name
        self.api_key = None
        try:
            from google.colab import userdata
            self.api_key = userdata.get(api_key_env_var)
        except ImportError:
            pass
        if not self.api_key:
            self.api_key = os.getenv(api_key_env_var)
        if not self.api_key:
            raise ValueError(
                f"API key for {self.__class__.__name__} (env var: {api_key_env_var}) not found."
            )

    def _construct_judge_prompt(self, transcript: str, response_a: str, response_b: str) -> str:
        return f"""
**CRITICAL INSTRUCTIONS FOR SOAP NOTE EVALUATION**
**Role:** You are an expert clinical documentation evaluator AI.
**Task:** Critically assess the quality of both AI-Generated SOAP Notes provided below and INDICATE the BEST SOAP NOTE. Your evaluation must compare BOTH Generated Notes against each other and the **Original Transcript**. Determine which SOAP Note is superior based on accuracy, completeness, structure, and conciseness, following the criteria below.

**FOUNDATIONAL PRINCIPLE: A SOAP Note must ONLY contain information EXPLICITLY stated in the transcript.**

**Evaluation Criteria (Weighting indicated):**

1.  **Accuracy & Fidelity to Transcript (MOST CRITICAL - 70% of total score)**
    *   The Generated Note must contain **ONLY** information **explicitly stated** in the Original Transcript.
    *   **AUTOMATIC FAILURE** for any of these severe violations:
        * Adding diagnoses not explicitly stated by the clinician
        * Including medication dosages not specifically mentioned
        * Attributing causes of conditions not directly stated
        * Creating follow-up plans not verbalized in the transcript
    *   **Penalize HEAVILY (-30 to -50 points)** for any other information in the Generated Note that is *NOT* found in the Original Transcript (hallucinations, inferences, external knowledge).
    *   Are facts (dosages, timelines, symptoms) represented exactly *as stated in the transcript*? Penalize inaccuracies (-5 to -15 points per significant error).
    *   Verify speaker attribution: Information must be attributed to the correct speaker (e.g., doctor's statements should not be presented as patient-reported).

2.  **Transcript Content Coverage & Completeness (SECONDARY CRITICALITY - 20% of total score)**
    *   Compare the Generated Note to the **Original Transcript**. Does it include all clinically significant information (chief complaint, key history, objective findings, assessment, plan) **explicitly present** in the transcript?
    *   **Penalize (-5 to -20 points per significant omission)** for omitting significant information *that was mentioned* in the transcript.
    *   **CRUCIAL:** Do **NOT** penalize for omitting information if it was **absent** from the Original Transcript.

3.  **Conciseness & Clarity (TERTIARY CRITICALITY - 10% of total score)**
    *   Is the language clear and concise? Penalize excessive verbosity or awkward phrasing (-2 to -5 points).
    *   Is the note organized in standard SOAP format with clear delineation between sections?
    *   Does the note use appropriate clinical terminology that was present in the transcript?

**EXPLICIT PROHIBITIONS:**
1. The SOAP note must NEVER include clinical judgments or assessments beyond what the clinician explicitly stated
2. The SOAP note must NEVER add specific medication instructions not verbalized in the transcript
3. The SOAP note must NEVER create treatment rationales that weren't explained by the clinician
4. The SOAP note must NEVER attribute statements to the wrong speaker

**Input:**
**Original Transcript:**
<input>
    <transcript>
        {transcript}
    </transcript>
    <responses>
        <response_A>
            {response_a}
        </response_A>
        <response_B>
            {response_b}
        </response_B>
    </responses>
</input>

For each response, provide a detailed analysis listing:
1. Any content that was NOT explicitly stated in the transcript (with severity rating: Critical, Major, Minor)
2. Any significant clinical information from the transcript that was omitted
3. Any structural or clarity issues

After your analysis, conclude your response with EITHER "CHOICE: A" OR "CHOICE: B" on a new line, indicating which response contains fewer and less severe inaccuracies.
"""

    def _parse_response(self, llm_output: str) -> Optional[str]:
        match = re.search(r"CHOICE:\s*([AB])", llm_output, re.IGNORECASE | re.MULTILINE)
        if match: return match.group(1).upper()

        llm_output_lower = llm_output.lower()
        if "response a is better" in llm_output_lower: return "A"
        if "response b is better" in llm_output_lower: return "B"

        log_output_display = llm_output
        if len(log_output_display) > 1000: # Increased length for more context in logs
            log_output_display = log_output_display + "... (truncated after 1000 chars)"
        else:
            log_output_display = llm_output # Ensure it does not exceed 1000
        print(f"Warning ({self.__class__.__name__}): Could not parse choice from LLM output. Raw LLM output snippet: '{log_output_display}'")
        return None

    def evaluate(self, transcript: str, response_a: str, response_b: str) -> Optional[str]:
        raise NotImplementedError("Subclasses must implement the evaluate method.")

class GeminiConcurrentUserEvaluator(BaseEvaluator):
    def __init__(self, api_key_env_var: str = "GEMINI_API_KEY", model_name: str = GEMINI_EVAL_MODEL_NAME):
        super().__init__(api_key_env_var, model_name)
        self.api_url = GEMINI_API_URL_TEMPLATE.format(model_name=self.model_name)
        print(f"GeminiConcurrentUserEvaluator initialized with model: {self.model_name}")

    def evaluate(self, transcript: str, response_a: str, response_b: str) -> Optional[str]:
        judge_prompt_text = self._construct_judge_prompt(transcript, response_a, response_b)
        headers = {"Content-Type": "application/json"}
        payload = {
            "contents": [{"parts": [{"text": judge_prompt_text}]}],
            "generationConfig": {
                "temperature": 0.2,
                "maxOutputTokens": 20000 # MODIFIED: Explicitly set maxOutputTokens
            }
        }
        response_obj = None
        try:
            full_api_url = f"{self.api_url}?key={self.api_key}"
            response_obj = requests.post(full_api_url, headers=headers, json=payload, timeout=120) # Increased timeout slightly
            response_obj.raise_for_status()

            if not response_obj.content:
                print(f"Error (GeminiConcurrentUserEvaluator): Empty response content from API for model {self.model_name}.")
                return None
            try:
                response_data = response_obj.json()
            except requests.exceptions.JSONDecodeError:
                print(f"Error (GeminiConcurrentUserEvaluator): Non-JSON response from API for model {self.model_name}. Status: {response_obj.status_code}. Response text: {response_obj.text[:500]}")
                return None

            if 'promptFeedback' in response_data:
                prompt_feedback = response_data['promptFeedback']
                if 'blockReason' in prompt_feedback:
                    block_reason = prompt_feedback.get('blockReason')
                    safety_ratings_feedback = prompt_feedback.get('safetyRatings', [])
                    print(f"Warning (GeminiConcurrentUserEvaluator): Prompt was blocked for model {self.model_name}. Reason: {block_reason}. Safety Feedback: {safety_ratings_feedback}. Full Feedback: {prompt_feedback}")
                    return None

            if 'candidates' not in response_data or not response_data['candidates']:
                err_msg = f"Error (GeminiConcurrentUserEvaluator): 'candidates' field missing or empty in response from model {self.model_name}."
                if 'promptFeedback' in response_data:
                     err_msg += f" Prompt Feedback: {response_data['promptFeedback']}"
                err_msg += f" Response snippet: {str(response_data)}"
                print(err_msg)
                return None

            candidate = response_data['candidates'][0]
            finish_reason = candidate.get('finishReason')
            token_count = candidate.get('tokenCount')
            safety_ratings_candidate = candidate.get('safetyRatings', []) # Ensure it's a list

            # Check for content and parts before accessing them
            if 'content' not in candidate or \
               'parts' not in candidate.get('content', {}) or \
               not isinstance(candidate['content'].get('parts'), list) or \
               not candidate['content']['parts']: # Check if parts list is non-empty
                # This case might happen if finishReason is SAFETY or other error states where content isn't generated
                print(f"Error (GeminiConcurrentUserEvaluator): 'content' or 'parts' structure malformed or empty in candidate from model {self.model_name}. "
                      f"Finish Reason: {finish_reason}. Token Count: {token_count}. Candidate Safety Ratings: {safety_ratings_candidate}. "
                      f"Full Candidate: {str(candidate)}")
                return None

            # Now safe to access parts[0]
            first_part = candidate['content']['parts'][0]
            if 'text' not in first_part:
                print(f"Error (GeminiConcurrentUserEvaluator): 'text' missing in first part of candidate content from model {self.model_name}. "
                      f"Finish Reason: {finish_reason}. Token Count: {token_count}. Candidate Safety Ratings: {safety_ratings_candidate}. "
                      f"Full Candidate: {str(candidate)}")
                return None

            llm_output = first_part['text']

            if not llm_output.strip():
                # This is the critical log for the "MAX_TOKENS but empty text" issue
                print(f"Warning (GeminiConcurrentUserEvaluator): Extracted LLM output is empty or whitespace for model {self.model_name}. "
                      f"Finish Reason: {finish_reason}. Token Count: {token_count}. Candidate Safety Ratings: {safety_ratings_candidate}. "
                      f"Full Candidate: {str(candidate)}")
                # If finishReason is MAX_TOKENS here, it's highly suspicious.
                # If finishReason is SAFETY, this is expected if text is empty.
                return None

            if finish_reason and finish_reason != "STOP":
                # Log if generation stopped for a reason other than "STOP" but still produced some text
                print(f"Warning (GeminiConcurrentUserEvaluator): Candidate finishReason is '{finish_reason}' (not 'STOP') for model {self.model_name}. "
                      f"Output might be incomplete. Token Count: {token_count}. Candidate Safety Ratings: {safety_ratings_candidate}. "
                      f"Full Candidate: {str(candidate)}")

            return self._parse_response(llm_output)

        except requests.exceptions.Timeout:
            print(f"Error (GeminiConcurrentUserEvaluator): Request timed out for model {self.model_name}.")
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error (GeminiConcurrentUserEvaluator): API request error for model {self.model_name}: {e}")
            if response_obj: print(f"Response status: {response_obj.status_code}, Content: {response_obj.text}")
            return None
        except (KeyError, IndexError, TypeError) as e:
            err_msg = f"Error (GeminiConcurrentUserEvaluator): API response parsing error (e.g., unexpected structure) for model {self.model_name}: {e}."
            if response_obj and hasattr(response_obj, 'text'):
                err_msg += f" Raw Response snippet: {response_obj.text}"
            else:
                err_msg += " No response object or text available."
            print(err_msg)
            return None
        except Exception as e:
            print(f"Error (GeminiConcurrentUserEvaluator): Unexpected error during evaluation for model {self.model_name}: {e}")
            return None

class DeepseekConcurrentUserEvaluator(BaseEvaluator):
    def __init__(self, api_key_env_var: str = "DEEPSEEK_API_KEY", model_name: str = DEEPSEEK_EVAL_MODEL_NAME):
        super().__init__(api_key_env_var, model_name)
        self.api_url = DEEPSEEK_API_URL
        print(f"DeepseekConcurrentUserEvaluator initialized with model: {self.model_name}")

    def evaluate(self, transcript: str, response_a: str, response_b: str) -> Optional[str]:
        judge_prompt_text = self._construct_judge_prompt(transcript, response_a, response_b)
        headers = {"Content-Type": "application/json", "Authorization": f"Bearer {self.api_key}"}
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": judge_prompt_text}],
            "temperature": 0.2,
            "max_tokens": 1024 # Ensuring this is present as per last correct version
        }
        response_obj = None
        try:
            response_obj = requests.post(self.api_url, headers=headers, json=payload, timeout=90)
            response_obj.raise_for_status()
            if not response_obj.content:
                print(f"Error (DeepseekConcurrentUserEvaluator): Empty response from API for model {self.model_name}.")
                return None
            try: response_data = response_obj.json()
            except requests.exceptions.JSONDecodeError:
                print(f"Error (DeepseekConcurrentUserEvaluator): Non-JSON response: {response_obj.text}")
                return None

            if "choices" not in response_data or not response_data["choices"] or \
               "message" not in response_data["choices"][0] or \
               "content" not in response_data["choices"][0]["message"]:
                 print(f"Error (DeepseekConcurrentUserEvaluator): Malformed response structure from model {self.model_name}. Response snippet: {str(response_data)}")
                 return None

            llm_output = response_data["choices"][0]["message"]["content"]
            if not llm_output.strip():
                print(f"Warning (DeepseekConcurrentUserEvaluator): Extracted LLM output is empty or whitespace for model {self.model_name}. Response snippet: {str(response_data)}")
                return None

            return self._parse_response(llm_output)
        except requests.exceptions.Timeout:
            print(f"Error (DeepseekConcurrentUserEvaluator): Request timed out for model {self.model_name}.")
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error (DeepseekConcurrentUserEvaluator): API request error for model {self.model_name}: {e}")
            if response_obj: print(f"Status: {response_obj.status_code}, Content: {response_obj.text}")
            return None
        except (KeyError, IndexError, TypeError) as e:
            print(f"Error (DeepseekConcurrentUserEvaluator): API response parsing error for model {self.model_name}: {e}.")
            if response_obj and hasattr(response_obj, 'text'): print(f"Response snippet: {response_obj.text}")
            return None
        except Exception as e:
            print(f"Error (DeepseekConcurrentUserEvaluator): Unexpected error for model {self.model_name}: {e}")
            return None

# --- Global list of evaluators & Initialization (Keep as before) ---
EVALUATORS_LIST: List[BaseEvaluator] = []
def initialize_evaluators():
    global EVALUATORS_LIST
    try:
        print("Initializing evaluators...")
        gemini_judge = GeminiConcurrentUserEvaluator()
        deepseek_judge = DeepseekConcurrentUserEvaluator()
        EVALUATORS_LIST = [gemini_judge, deepseek_judge]
        if not EVALUATORS_LIST:
            print("Warning: EVALUATORS_LIST is empty after initialization attempt.")
        else:
            print(f"Successfully initialized {len(EVALUATORS_LIST)} evaluators.")
    except ValueError as e:
        print(f"ERROR during evaluator initialization (API key issue?): {e}")
        EVALUATORS_LIST = []
    except Exception as e:
        print(f"UNEXPECTED ERROR during evaluator initialization: {e}")
        EVALUATORS_LIST = []


# --- `run_comparison` function (remains the same) ---
def run_comparison(transcript: str, final_response_A: str, final_response_B: str, pair_identifier: str) -> List[float]:
    global EVALUATORS_LIST
    votes = {"A": 0, "B": 0, "Error": 0}
    judge_decisions_log = {}

    if not EVALUATORS_LIST:
        print(f"Pair {pair_identifier}: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]")
        return [0.0, 0.0]

    print(f"\n--- Evaluating Pair {pair_identifier} (Evaluators called concurrently) ---")
    # Truncate print output for brevity in logs
    print(f"Transcript: {transcript if len(transcript) > 100 else transcript}...")
    print(f"Response A: {final_response_A if len(final_response_A) > 100 else final_response_A}...")
    print(f"Response B: {final_response_B if len(final_response_B) > 100 else final_response_B}...")


    with concurrent.futures.ThreadPoolExecutor(max_workers=len(EVALUATORS_LIST), thread_name_prefix=f"Pair{pair_identifier}Eval_") as executor:
        future_to_evaluator = {
            executor.submit(evaluator.evaluate, transcript, final_response_A, final_response_B): evaluator
            for evaluator in EVALUATORS_LIST
        }

        for future in concurrent.futures.as_completed(future_to_evaluator):
            evaluator_instance = future_to_evaluator[future]
            eval_class_name = evaluator_instance.__class__.__name__
            try:
                decision = future.result()
                print(f"Pair {pair_identifier}: Result from {eval_class_name} ({evaluator_instance.model_name}) -> {decision}")
                judge_decisions_log[eval_class_name] = decision

                if decision == "A":
                    votes["A"] += 1
                elif decision == "B":
                    votes["B"] += 1
                else:
                    votes["Error"] += 1
            except Exception as exc:
                print(f"!! EXCEPTION processing result for Pair {pair_identifier} from {eval_class_name}: {exc}")
                votes["Error"] += 1
                judge_decisions_log[eval_class_name] = f"Exception: {exc}"

    print(f"\nPair {pair_identifier}: Final Votes --- A: {votes['A']}, B: {votes['B']}, Error/Undecided: {votes['Error']}")

    pair_specific_rewards = [0.0, 0.0]
    num_effective_evaluators = len(EVALUATORS_LIST) - votes["Error"]

    if num_effective_evaluators == 0:
        print(f"Pair {pair_identifier}: All evaluators failed or could not decide. Neutral rewards assigned.")
    elif votes["A"] > 0 and votes["A"] == num_effective_evaluators and votes["B"] == 0:
        print(f"Pair {pair_identifier}: All responding judges unanimously chose A. Rewards: [1.0, -1.0]")
        pair_specific_rewards = [1.0, -1.0]
    elif votes["B"] > 0 and votes["B"] == num_effective_evaluators and votes["A"] == 0:
        print(f"Pair {pair_identifier}: All responding judges unanimously chose B. Rewards: [-1.0, 1.0]")
        pair_specific_rewards = [-1.0, 1.0]
    else:
        print(f"Pair {pair_identifier}: Judges did not unanimously agree among responders, or errors prevented unanimity. Neutral rewards assigned.")

    return pair_specific_rewards

# --- run_evaluation_step (remains the same) ---
def run_evaluation_step(prompts: List[List[Dict[str, str]]],
                        completions: List[List[Dict[str, str]]],
                        reference_unused: Any = None,
                        max_concurrent_pairs: int = 4,
                        **kwargs) -> List[float]:
    all_rewards_flat_ordered = [0.0] * len(completions)

    if not (isinstance(prompts, list) and isinstance(completions, list) and
            len(prompts) == len(completions) and len(prompts) % 2 == 0):
        print("run_evaluation_step: Invalid 'prompts' or 'completions' structure. Returning neutral rewards.")
        return all_rewards_flat_ordered

    num_pairs = len(prompts) // 2
    if num_pairs == 0:
        return []

    print(f"\nStarting evaluation of {num_pairs} pairs concurrently (max_concurrent_pairs={max_concurrent_pairs})...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent_pairs, thread_name_prefix="PairProcessor_") as pair_executor:
        future_to_pair_index = {}

        for i in range(num_pairs):
            prompt_idx = i * 2
            pair_identifier = f"OrigPrompt{i+1}"

            if not (prompts[prompt_idx] and isinstance(prompts[prompt_idx], list) and prompts[prompt_idx][0] and
                    'content' in prompts[prompt_idx][0] and
                    completions[prompt_idx] and isinstance(completions[prompt_idx], list) and completions[prompt_idx][0] and
                    'content' in completions[prompt_idx][0] and
                    completions[prompt_idx+1] and isinstance(completions[prompt_idx+1], list) and completions[prompt_idx+1][0] and
                    'content' in completions[prompt_idx+1][0]):
                print(f"run_evaluation_step: Invalid data structure for pair {pair_identifier}. Assigning neutral rewards.")
                all_rewards_flat_ordered[prompt_idx] = 0.0
                all_rewards_flat_ordered[prompt_idx + 1] = 0.0
                continue

            prompt_content = prompts[prompt_idx][0]['content']
            transcript_match = re.search(r'--- TRANSCRIPT START ---\s*(.*?)\s*--- TRANSCRIPT END ---',
                                        prompt_content, re.DOTALL)
            transcript = transcript_match.group(1).strip() if transcript_match else prompt_content.strip()

            completion_content_A = completions[prompt_idx][0]['content']
            completion_content_B = completions[prompt_idx + 1][0]['content']
            final_response_A = re.sub(r'<think>.*?</think>', '', completion_content_A, flags=re.DOTALL).strip()
            final_response_B = re.sub(r'<think>.*?</think>', '', completion_content_B, flags=re.DOTALL).strip()

            future = pair_executor.submit(run_comparison, transcript, final_response_A, final_response_B, pair_identifier)
            future_to_pair_index[future] = i

        for future in concurrent.futures.as_completed(future_to_pair_index):
            original_pair_idx = future_to_pair_index[future]
            rewards_idx_start = original_pair_idx * 2
            try:
                pair_rewards_result = future.result()
                all_rewards_flat_ordered[rewards_idx_start] = pair_rewards_result[0]
                all_rewards_flat_ordered[rewards_idx_start + 1] = pair_rewards_result[1]
            except Exception as exc:
                print(f"!! EXCEPTION processing pair {original_pair_idx+1} in run_evaluation_step: {exc}")
                all_rewards_flat_ordered[rewards_idx_start] = 0.0
                all_rewards_flat_ordered[rewards_idx_start + 1] = 0.0

    print("\nFinished concurrent evaluation of all pairs.")
    return all_rewards_flat_ordered

# # --- Example Usage (Illustrative - remains the same) ---
# if __name__ == "__main__":
#     # IMPORTANT: Set your API keys as environment variables or Colab secrets
#     # os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_KEY"
#     # os.environ["DEEPSEEK_API_KEY"] = "YOUR_DEEPSEEK_KEY"
#     initialize_evaluators()

#     if not EVALUATORS_LIST:
#         print("Cannot run example: Evaluators failed to initialize. Ensure API keys are set.")
#     else:
#         print("\n--- Running Example Evaluation for Multiple Pairs Concurrently ---")
#         # Using more detailed transcripts for better testing
#         transcript1_content = "--- TRANSCRIPT START ---\nPatient A, 67-year-old male, presents with a persistent dry cough for 3 weeks, worsening at night. Reports low-grade fever (37.8 C) for the past 5 days and general malaise. Denies chest pain or shortness of breath. Smoker, 1 pack per day for 40 years. Vitals: T 37.8C, P 88, RR 18, BP 130/80. Lungs: Occasional wheezes bilaterally.\n--- TRANSCRIPT END ---"
#         transcript2_content = "--- TRANSCRIPT START ---\nPatient B, 25-year-old female, twisted her right ankle playing soccer 2 days ago. Reports immediate pain, swelling, and difficulty bearing weight. Rates pain 7/10. Ibuprofen provides minimal relief. No numbness or tingling. On exam: Moderate edema and ecchymosis over lateral malleolus. Point tenderness over ATFL. Ottawa ankle rules positive for X-ray. ROM limited by pain.\n--- TRANSCRIPT END ---"
#         transcript3_content = "--- TRANSCRIPT START ---\nPatient C, 42-year-old male, complains of a throbbing headache for 2 days, localized to the frontal region, 8/10 intensity. Associated with photophobia and nausea. No vomiting. Reports similar headaches 1-2 times a year, usually resolves with rest. Denies fever, neck stiffness, or recent trauma. Neurological exam is grossly intact.\n--- TRANSCRIPT END ---"

#         example_prompts_for_trainer = [
#             [{'role': 'user', 'content': transcript1_content}], [{'role': 'user', 'content': transcript1_content}],
#             [{'role': 'user', 'content': transcript2_content}], [{'role': 'user', 'content': transcript2_content}],
#             [{'role': 'user', 'content': transcript3_content}], [{'role': 'user', 'content': transcript3_content}],
#         ]
#         # More distinct example completions
#         example_completions_for_trainer = [
#             [{'role': 'assistant', 'content': "S: 67yo M with 3wk dry cough, worse at night. 5d low fever (37.8C), malaise. No CP/SOB. Smoker (1ppd x 40yrs). O: T37.8, P88, RR18, BP130/80. Lungs: Occasional wheezes bilaterally. A: Acute bronchitis, consider ruling out pneumonia given smoking history and fever. P: Chest X-ray. Recommend smoking cessation. Symptomatic treatment: antitussives, hydration. F/u if worsens or no improvement in 1 week."}], # Good A
#             [{'role': 'assistant', 'content': "S: Cough and fever. O: Wheezing. A: Bronchitis. P: Antibiotics."}], # Bad B (hallucinates antibiotics, too brief)
#             [{'role': 'assistant', 'content': "S: Ankle injury from soccer. O: Swollen, tender ankle. A: Ankle sprain. P: RICE therapy. X-ray to rule out fracture."}], # Okay A
#             [{'role': 'assistant', 'content': "S: 25 F, R ankle pain x2d after soccer. Pain 7/10, swelling, cannot bear weight. Ibuprofen not helping. No paresthesias. O: Mod edema, ecchymosis lateral malleolus. ATFL tender. Ottawa rules positive. ROM limited. A: R ankle sprain, likely ATFL. R/O fracture. P: X-ray R ankle. RICE. Crutches. Stronger analgesia (e.g., naproxen). F/U with X-ray results."}], # Excellent B
#             [{'role': 'assistant', 'content': "S: Headache, photophobia, nausea. History of similar HAs. O: Normal neuro exam. A: Migraine. P: Sumatriptan. Rest."}], # Good A
#             [{'role': 'assistant', 'content': "S: Headache. O: Normal. A: Tension headache. P: Tylenol."}], # Okay B (less accurate assessment based on photophobia/nausea)
#         ]

#         rewards_from_eval = run_evaluation_step(
#             prompts=example_prompts_for_trainer,
#             completions=example_completions_for_trainer,
#             max_concurrent_pairs=3
#         )
#         print(f"\nExample Rewards Returned (flat list): {rewards_from_eval}")

In [ ]:
# !pip uninstall -y vllm
# !pip install vllm==v0.8.5.post1
from google.colab import userdata
import os
os.environ["VLLM_USE_V1"] = "0"
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY') # Also set env var
# Use f-string to pass the key value to the shell command
key = userdata.get('WANDB_API_KEY')
!wandb login {key}

# Set environment variables
os.environ["WANDB_PROJECT"] = "AI-Scribing"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
import sys
import asyncio
import torch
import functools
import time
import traceback
import threading
import gc
import queue
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from unsloth import FastModel, FastLanguageModel
from unsloth.chat_templates import get_chat_template
import re
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer
from transformers import TrainingArguments, GenerationConfig # GRPOConfig inherits from this
# from vllm import LLM, SamplingParams
import google.generativeai as genai
from google.colab import userdata

# === Core Configuration ===
MAX_SEQ_LENGTH = 7000 # Reduced for simplicity, adjust if needed
LOAD_IN_4BIT = True
TARGET_DTYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
OUTPUT_DIR = "/content/drive/MyDrive/Models/outputs_grpo_minimal-qwen-tung-tung-tung-tung-sahur"

# GRPO Params
MAX_STEPS = 100 # Reduced for faster minimal run
LEARNING_RATE = 5e-6
NUM_GENERATIONS = 8 # Number of completions per prompt (keep low for speed)
GRAD_ACCUMULATION_STEPS = 1
MAX_PROMPT_LENGTH = 5000
MAX_COMPLETION_LENGTH = MAX_SEQ_LENGTH - MAX_PROMPT_LENGTH

# LoRA Params (Simplified)
LORA_R = 64
LORA_ALPHA = 128

# === Load Model & Tokenizer ===
MODEL_NAME = "Qwen/Qwen3-4B"
print(f"Loading model: {MODEL_NAME}")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME, #"Qwen/Qwen3-4B"
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=LOAD_IN_4BIT,
    dtype=TARGET_DTYPE,
    # fast_inference=True, # Enable vLLM fast inference

    # token = "hf_...", # Add HF token if model is gated
)

print("Applying Chat Template...")
# Update chat template for Qwen3
try:
    tokenizer = get_chat_template(tokenizer, chat_template="qwen")
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
except:
    print("Warning: Failed to apply specific chat template. Using default.")
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
tokenizer.qwen_enable_thinking = True

print("Applying PEFT (LoRA)...")
model = FastModel.get_peft_model(
    model,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    # target_modules automatically detected by Unsloth for Gemma
    lora_dropout=0.05,
    bias="none",
    modules_to_save=None,  # Don't save any extra modules
    use_gradient_checkpointing="unsloth",  # Disable for higher GPU utilization

)
model.print_trainable_parameters()

# === Prepare Dataset ===
def format_examples_for_grpo(examples):
    prompts = []
    references = []
    transcripts = []
    for dialogue, note in zip(examples["dialogue"], examples["augmented note"]):
        if dialogue and note:
            user_content = f"""
**Role:** You are an AI assistant specialized in generating clinical SOAP notes.

**Task:** Generate a concise, accurate, and clinically relevant SOAP note based **STRICTLY AND SOLELY** on the provided doctor-patient interaction transcript.

**CRITICAL INSTRUCTIONS:**

1.  **Strict Transcript Adherence:** Generate the SOAP note using **ONLY** information **explicitly stated** within the provided transcript.
2.  **NO Assumptions or External Knowledge:** **DO NOT** infer information, add details not mentioned (even if clinically likely), make assumptions, or use external medical knowledge. Adherence to the transcript is paramount.
3.  **Standard SOAP Structure:** Organize the output clearly into the following sections using **EXACTLY** these headings:
    *   **S – Subjective**
    *   **O – Objective**
    *   **A – Assessment**
    *   **P – Plan**
4.  **NO Extraneous Text:** The output must contain **ONLY** the four section headings (S, O, A, P) and the corresponding content derived *directly* from the transcript. **DO NOT** include introductory sentences (e.g., "Here is the SOAP note:"), concluding remarks, disclaimers, notes about the generation process, metadata, or *any* other text before, between, or after the S/O/A/P sections.

**Formatting:**

*   Use clear headings for each SOAP section (as listed above).
*   Be concise but ensure all relevant details *from the transcript* are included under the correct heading.
*   Use standard medical abbreviations only if they are unambiguous and directly supported by the transcript's terminology.

**Input:** You will receive a doctor-patient transcript.

**Output:** Generate **ONLY** the structured SOAP note (S/O/A/P sections and content) based on the critical instructions above.

--- TRANSCRIPT START ---
{dialogue}
--- TRANSCRIPT END ---

SOAP NOTE:
"""  # Note the explicit ending to prompt the model to start generating

            prompt_messages = [
                {"role": "user", "content": user_content},
            ]
            prompts.append(prompt_messages)
            references.append(note)
            transcripts.append(dialogue)
    return {"prompt": prompts, "reference": references, "transcript": transcripts}

# === Load and Process Dataset ===
print("Loading dataset...")
# Load the ClinicianFOCUS/ACI-Bench-Refined dataset from Hugging Face
dataset = load_dataset("ClinicianFOCUS/ACI-Bench-Refined")
print(f"Dataset loaded with {len(dataset['train'])} training examples")

# Process the dataset using your formatting function
raw_processed = format_examples_for_grpo(dataset["train"])

# Convert to a list-based dataset compatible with indexing
from torch.utils.data import Dataset

class GRPODataset(Dataset):
    def __init__(self, prompts, references, transcripts): # <-- Add transcripts
        self.prompts = prompts
        self.references = references
        self.transcripts = transcripts # <-- Store transcripts

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        return {
            "prompt": self.prompts[idx],
            "reference": self.references[idx],
            "transcript": self.transcripts[idx] # <-- Return transcript
        }

# When creating the dataset:
raw_processed = format_examples_for_grpo(dataset["train"])
processed_dataset = GRPODataset(
    raw_processed["prompt"],
    raw_processed["reference"],
    raw_processed["transcript"] # <-- Pass transcripts
)
print(f"Processed {len(processed_dataset)} examples for training")

# === Setup GRPO Training ===
reward_functions = [run_evaluation_step]
reward_weights_list = [1.0]

generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
    top_k=20,
    max_new_tokens=MAX_COMPLETION_LENGTH, # Set explicitly here too
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True
)
model.generation_config = generation_config
print("GenerationConfig attached.")

# --- Corrected GRPOConfig ---
training_args = GRPOConfig(
    weight_decay=0.01,
    max_grad_norm=1.0,
    # use_vllm=True,
    dataloader_drop_last=True,  # Prevent partial batches that might have irregular memory patterns
    dataloader_num_workers=1,   # Reduce parallel data loading
    output_dir=OUTPUT_DIR,
    learning_rate=LEARNING_RATE,
    max_steps=MAX_STEPS,
    per_device_train_batch_size=NUM_GENERATIONS, # GRPO requires this
    gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
    gradient_checkpointing=True,
    logging_steps=1,
    save_steps=20,
    optim="adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    # GRPO specific args
    num_generations=NUM_GENERATIONS,
    max_prompt_length=MAX_PROMPT_LENGTH,
    max_completion_length=MAX_COMPLETION_LENGTH,
    beta=0.15, # Default GRPO beta
    reward_weights=reward_weights_list, # <-- CORRECT PARAMETER LOCATION
    # Technical args
    bf16=TARGET_DTYPE == torch.bfloat16,
    fp16=TARGET_DTYPE == torch.float16,
    report_to="wandb", # "tensorboard"
    run_name="Tung-Tung-Tung-Tung-Sahur", # Optional: Changed run name
    remove_unused_columns=False, # Important for GRPO
    seed=42,
    # use_vllm=True,
)

# --- Corrected GRPOTrainer ---
# Note: We rely on the trainer implicitly passing kwargs to the reward functions
# or the reward function accessing the global `gemini_model`.
# If this fails, using functools.partial would be the next step.
trainer = GRPOTrainer(
    model=model,
    tokenizer=tokenizer, # Passing tokenizer is often implicitly handled
    args=training_args,
    train_dataset=processed_dataset,
    reward_funcs=reward_functions, # <-- CORRECT PARAMETER NAME
    # reward_weights removed from here
    # reward_kwargs removed from here
)

# === Start Training ===
print("Starting GRPO training...")
trainer.train(resume_from_checkpoint=True)
print("✅ Training finished.")

HF_REPO_ID = "ClinicianFOCUS/Clinician-Note-1.1a"

# === Save Final Adapters ===
final_adapter_path = os.path.join(OUTPUT_DIR, "final_adapters")
try:
    print(f"\nSaving final LoRA adapters to {final_adapter_path}...")
    trainer.save_model(final_adapter_path) # Saves adapter config and weights
    # Trainer might not save tokenizer automatically, let's save it too
    tokenizer.save_pretrained(final_adapter_path)
    print(f"✅ Adapters and tokenizer saved to {final_adapter_path}")
except Exception as save_e:
    print(f"❌ Failed to save final adapters: {save_e}", file=sys.stderr)
    # Continue to merging attempt if possible, but adapters might be missing/corrupt

# === Merge and Save Final 16-bit Model using standard transformers ===
# Only proceed if training seemed to complete and adapters were likely saved
if 'final_adapter_path' in locals() and os.path.exists(os.path.join(final_adapter_path, "adapter_config.json")):
    print("\nMerging adapters and saving final 16-bit model...")

    # Clean up memory from training if possible (optional, but good practice)
    print("Cleaning up training resources...")
    del trainer # Remove trainer object
    if torch.cuda.is_available():
        torch.cuda.empty_cache() # Clear cached memory

    try:
        # Import standard transformers instead of using FastModel (Unsloth)
        from transformers import AutoModelForCausalLM, AutoTokenizer
        from peft import PeftModel

        print(f"Reloading base model {MODEL_NAME} in {TARGET_DTYPE} for merging...")

        # Load the base model with standard transformers
        base_model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.bfloat16 if TARGET_DTYPE == "bfloat16" else torch.float16,
            trust_remote_code=True,
            # token=os.environ.get('HF_TOKEN'), # Uncomment if needed
        )

        # Load the standard tokenizer
        std_tokenizer = AutoTokenizer.from_pretrained(
            MODEL_NAME,
            trust_remote_code=True,
            # token=os.environ.get('HF_TOKEN'), # Uncomment if needed
        )

        print(f"Loading trained adapters from {final_adapter_path}...")
        # Load the adapters using PEFT
        model = PeftModel.from_pretrained(base_model, final_adapter_path)

        # Merge the adapters into the base model
        print("Merging LoRA adapters...")
        merged_model = model.merge_and_unload()

        # Define path for the final merged model
        merged_save_path = os.path.join(OUTPUT_DIR, "final_merged_16bit")
        print(f"Saving merged 16-bit model to {merged_save_path}...")
        os.makedirs(merged_save_path, exist_ok=True)

        # Save the merged model and tokenizer
        merged_model.save_pretrained(merged_save_path, safe_serialization=True)
        std_tokenizer.save_pretrained(merged_save_path)

        print(f"✅ Merged 16-bit model saved successfully to {merged_save_path}")

        # === Push model to Hugging Face Hub ===
        print("\nPushing model to Hugging Face Hub...")

        try:
            # Set repository name for the model on HF Hub
            HF_REPO_ID = os.environ.get("HF_REPO_ID")
            if not HF_REPO_ID:
                print("Warning: HF_REPO_ID environment variable not set.")
                HF_REPO_ID = input("Enter your Hugging Face repository ID (username/repo-name): ")

            print(f"Pushing merged model to {HF_REPO_ID}...")

            # Push the model to HF Hub
            merged_model.push_to_hub(
                repo_id=HF_REPO_ID,
                commit_message="Push final merged fine-tuned model",
                private=False  # Set to True if you want a private repo
            )

            # Push tokenizer to HF Hub
            std_tokenizer.push_to_hub(
                repo_id=HF_REPO_ID,
                commit_message="Push tokenizer for fine-tuned model",
                private=False  # Set to True if you want a private repo
            )

            print(f"✅ Model and tokenizer successfully pushed to Hugging Face Hub: https://huggingface.co/{HF_REPO_ID}")

        except Exception as push_e:
            print(f"❌ Failed to push model to Hugging Face Hub: {push_e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)
            print("You can manually push the model later using the Hugging Face CLI:")
            print(f"  huggingface-cli upload {merged_save_path} {HF_REPO_ID}")

    except Exception as merge_e:
        print(f"❌ Failed during merging or saving the 16-bit model: {merge_e}", file=sys.stderr)
        traceback.print_exc(file=sys.stderr)

        # Fall back to pushing just the adapters if merging fails
        print("\nFalling back to pushing just the adapters...")
        try:
            from huggingface_hub import HfApi

            # Set repository name for the model on HF Hub
            HF_REPO_ID = os.environ.get("HF_REPO_ID")
            if not HF_REPO_ID:
                print("Warning: HF_REPO_ID environment variable not set.")
                HF_REPO_ID = input("Enter your Hugging Face repository ID (username/repo-name): ")

            print(f"Pushing adapters to {HF_REPO_ID}...")

            # Create an instance of HfApi
            api = HfApi()

            # Create the repository if it doesn't exist
            api.create_repo(
                repo_id=HF_REPO_ID,
                exist_ok=True,
                private=False  # Set to True if you want a private repo
            )

            # Push the adapters to HF Hub using upload_folder
            api.upload_folder(
                folder_path=final_adapter_path,
                repo_id=HF_REPO_ID,
                commit_message="Push LoRA adapters for fine-tuned model"
            )

            print(f"✅ Adapters successfully pushed to Hugging Face Hub: https://huggingface.co/{HF_REPO_ID}")
            print(f"Note: These are LoRA adapters that should be applied to the base model: {MODEL_NAME}")

        except Exception as adapter_push_e:
            print(f"❌ Failed to push adapters to Hugging Face Hub: {adapter_push_e}", file=sys.stderr)
            traceback.print_exc(file=sys.stderr)
else:
    print("\nSkipping merge and save of 16-bit model because adapter saving failed or training did not complete.")

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
Loading model: Qwen/Qwen3-4B
==((====))==  Unsloth 2025.4.8: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Applying Chat Template...
Applying PEFT (LoRA)...
Unsloth: Making `model.base_model.model.model` require gradients
trainable params: 132,120,576 || all params: 4,154,588,672 || trainable%: 3.1801
Loading dataset...
Dataset loaded with 207 training examples
Processed 207 examples for training
GenerationConfig attached.
Starting GRPO t

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 207 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 132,120,576/4,000,000,000 (3.30% trained)
/usr/local/lib/python3.11/dist-packages/unsloth/kernels/utils.py:438: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 2560], which does not match the required output shape [8, 1, 2560]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:30.)
  out = torch_matmul(X, W.t(), out = out)
/usr/local/lib/python3.11/dist-packages/unsloth/kernels/utils.py:443: User


Starting evaluation of 4 pairs concurrently (max_concurrent_pairs=4)...
Pair OrigPrompt1: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]
Pair OrigPrompt2: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]Pair OrigPrompt3: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]
Pair OrigPrompt4: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]


Finished concurrent evaluation of all pairs.


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / run_evaluation_step
81,0.000400,0.000000,0.000000,705.000000,0.002428,0.000000
82,0.000400,0.000000,0.000000,938.875000,0.002871,0.000000
83,0.000300,0.000000,0.000000,1075.375000,0.001825,0.000000
84,0.000400,0.000000,0.000000,997.250000,0.002443,0.000000
85,0.000400,0.000000,0.000000,756.750000,0.002734,0.000000
86,0.000300,0.000000,0.000000,917.000000,0.002267,0.000000
87,0.000300,0.000000,0.000000,1112.375000,0.002316,0.000000
88,0.000400,0.000000,0.000000,694.125000,0.002405,0.000000
89,0.000300,0.000000,0.000000,1074.750000,0.002031,0.000000
90,0.000500,0.000000,0.000000,753.875000,0.003224,0.000000



Starting evaluation of 4 pairs concurrently (max_concurrent_pairs=4)...
Pair OrigPrompt1: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]
Pair OrigPrompt2: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]Pair OrigPrompt3: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]Pair OrigPrompt4: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]



Finished concurrent evaluation of all pairs.

Starting evaluation of 4 pairs concurrently (max_concurrent_pairs=4)...
Pair OrigPrompt1: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]
Pair OrigPrompt2: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]Pair OrigPrompt3: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]
Pair OrigPrompt4: No evaluators available in EVALUATORS_LIST. Assigning neutral rewards [0.0, 0.0]


Finished concur

...

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✅ Model and tokenizer successfully pushed to Hugging Face Hub: https://huggingface.co/ClinicianFOCUS/Clinician-Note-2.0a
